<a href="https://colab.research.google.com/github/Cerino-rigo/EC3002C.602-2023/blob/main/RegresionLinealMultiple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # **Modelos supervisados: Regresión Lineal Multiple con `sklearn`**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## **Bibliotecas**

In [ ]:
# Operaciones matemáticas y estadísticas
import pandas as pd
import numpy as np

In [ ]:
# Visualización
import plotly.express as px
import plotly.graph_objs as go

## **Conjunto de Datos**

In [ ]:
url = "/content/drive/MyDrive/Machine Learning/Ecommerce_Customers.csv"

In [ ]:
datos = pd.read_csv(url)

In [ ]:
datos.head()

<p align="justify">
Es una empresa de comercio electrónico con sede en la ciudad de Nueva York que vende ropa en línea, pero también tienen sesiones de asesoramiento sobre estilo y ropa en la tienda. Los clientes , tienen sesiones/reuniones con un estilista personal, luego pueden ordenar a través de una aplicación móvil o sitio web la ropa que desean.
<br>
<br>
Intentan identificar si debe enfocarse en mejorar su experiencia de aplicación movil o en su página web.
<br>
<br>
Este es un conjunto de datos de los clientes de la empresa. El cual tiene información del cliente, como correo electrónico, dirección postal y su color de avatar. También tiene columnas de valores numéricos:

* **Avg. Session Length**: duración promedio de sesiones de asesoramiento de estilo en la tienda.

* **Time on App**: tiempo promedio de permanencia en la aplicación en minutos.

* **Time on Website**: tiempo promedio de permanencia en el sitio web en minutos.

* **Length of Membership**: cuántos años ha sido miembro el cliente.

* **Yearly Amount Spent**: monto anual gastado en la plataforma.





In [ ]:
datos.info()

<p align="justify">
En la biblioteca <code>Pandas</code>, el método <code>unique()</code> se utiliza para obtener los valores únicos presentes en una serie o columna de un DataFrame. El objetivo es revisar que no hayan clientes repetidos.  

In [ ]:
len(datos.Email.unique())

<p align="justify">
Con el método <code>drop()</code> de <code>Pandas</code> se eliminan las columnas del <code>DataFrame</code> que ya no son útiles. Descartamos <code>Avatar</code>, <code>Email</code> y <code>Address</code>.

In [ ]:
datos = datos.drop(["Avatar","Email","Address"], axis=1)

<p align="justify">
Se renombran las columnas para asignarles nombres más descriptivos y significativos. Es posible utilizar el método <code>rename()</code>.

In [ ]:
# Renombramos columnas:
datos.rename(columns={"Avg. Session Length": "Tiempo_sesión",
                      "Time on App":"Tiempo_app",
                      "Time on Website":"Tiempo_web",
                      "Length of Membership":"Años_miembro",
                      "Yearly Amount Spent":"Gasto_anual"},inplace=True)

In [ ]:
datos.head()

## **Análisis exploratorio de datos (Estadística Descriptiva)**

<p align="justify">
Primero exploremos estos datos, antes de realizar la regresión lineal para tomar una decisión.
<br>
<br>
Solo utilizaremos datos numéricos.

In [ ]:
datos.info()

In [ ]:
datos.describe().T.round(3)

### **Distribución de los datos de la variable respuesta (`Gasto_anual`)**

En esta línea, se importa la función norm de la biblioteca scipy.stats. La función **norm** representa una distribución normal.

In [ ]:
from scipy.stats import norm

In [ ]:
#Distribución teórica
mu, sigma = norm.fit(datos.Gasto_anual)
print(mu)
print(sigma)

**mu**: Representa la media de la distribución normal ajustada a los datos.
**sigma**: Representa la desviación estándar de la distribución normal ajustada a los datos.
La función **fit** toma los datos de la columna Gasto_anual de datos y ajusta una distribución normal a esos datos, devolviendo la media (**mu**) y la desviación estándar (**sigma**) de la distribución ajustada.

In [ ]:
x = np.linspace(datos.Gasto_anual.min(),
                datos.Gasto_anual.max(),
                num = 100)
y = norm.pdf(x, mu, sigma)

In [ ]:
fig = go.Figure([go.Scatter(x = x,
                            y = y,
                            line = {"width":3},
                            name = "Función Densidad de Probabilidad Teórica (Normal)"),

                 go.Histogram(x=datos.Gasto_anual,
                              histnorm = "probability density",
                              name = "Distribución Real")])

fig.update_layout(template = "simple_white",
                  title = "Distribución del Gasto Anual",
                  )

fig.show()

Podemos observar que el gasto anual (variable respuesta) está normalmente distribuido.

### **Matriz de correlación**

<p align="justify">
La función <code>pearsonr</code> de la biblioteca <code>scipy.stats</code> en Python se utiliza para calcular el <b>coeficiente de correlación de Pearson</b> entre dos variables. El coeficiente de correlación de Pearson es una medida estadística que evalúa la fuerza y dirección de la relación lineal entre dos variables continuas. El resultado de pearsonr es un par de valores: el <b>coeficiente de correlación</b> y el <b>p-valor</b> asociado.

- El **coeficiente de correlación** de Pearson varía entre $-1$ y $1$. Un valor de $1$ indica una correlación positiva perfecta, lo que significa que las dos variables están perfectamente relacionadas en una relación lineal positiva. Un valor de $-1$ indica una correlación negativa perfecta, lo que implica una relación lineal negativa perfecta entre las variables. Un valor cercano a $0$ indica una correlación débil o inexistente entre las variables.

- El **p-valor** asociado proporciona una medida de la significancia estadística de la correlación calculada. Si el p-valor es menor que un umbral  predefinido (generalmente $0.05$), se considera que la correlación es estadísticamente significativa.

In [ ]:
from scipy.stats import pearsonr

In [ ]:
correlacion = pearsonr(x = datos.Tiempo_app, y =  datos.Gasto_anual)
print("")
print("Coeficiente de correlación de Pearson: {}".format(round(correlacion[0],4)))
print("P-valor: {}".format(correlacion[1]))

<p align="justify">
El test de correlación muestran una <b>relación lineal positiva de intensidad media</b> (r = 0.5) y <b>estadísticamente significativa</b> (p-valor = 6.905842369973249e-33).

<p align="justify">
El método <code>corr()</code> en <code>Pandas</code> se utiliza para calcular la matriz de correlación entre las variables numéricas de un DataFrame. Esta matriz muestra las correlaciones entre pares de variables y es útil para analizar la relación lineal entre diferentes variables en un conjunto de datos.

In [ ]:
corr_matrix = round(datos.corr(),3)
corr_matrix

<p align="justify">
El <b>Mapa de calor</b> o <b>Heatmap</b> es una herramienta gráfica utilizada para visualizar y analizar la relación entre variables en un conjunto de datos.

Tradicionalmente, la biblioteca [Seaborn](https://seaborn.pydata.org/generated/seaborn.heatmap.html) ha sido comúnmente empleada para generar Mapas de calor.

No obstante, en esta ocasión hemos optado por utilizar la biblioteca [Plotly](https://plotly.com/python/imshow/) para llevar a cabo esta representación gráfica.



In [ ]:
px.imshow(corr_matrix,
          title = "Matriz de Correlacion",
          text_auto=True,
          color_continuous_scale='fall',
          labels={"color":"Indice"})

<p align="justify">
A primera vista, se evidencia una relación fuerte entre la variable explicativa <code>Años_miembro</code> y la variable <code>Gasto_anual</code>. Asimismo, se puede apreciar una relación moderada entre la variable explicativa <code>Tiempo_app</code> y la variable <code>Gasto_anual</code>. La relación entre <code>Tiempo_web</code> y <code>Gasto_anual</code> es nula.
<br>
<br>
No obstante, no se observa una correlación significativa entre las variables explicativas entre sí. Esto indica que las variables explicativas son <b>independientes</b> entre sí y no hay una relación directa o sistemática entre ellas en el conjunto de datos analizado. Cada una de ellas puede proporcionar información única y no redundante para explicar la variable respuesta (<code>Gasto_anual</code>)

### **Scatterplot de cada variable explicativa**

In [ ]:
X = datos[['Tiempo_sesión', 'Tiempo_app','Tiempo_web','Años_miembro']]
y = datos['Gasto_anual']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
for i in X.columns:
  fig = px.scatter(datos,
             x = i,
             y = y,
             trendline="ols",
             trendline_color_override="darkorange",
             template = "gridon",
             title = i)
  fig.show()

 ## **Estandarización de variables numéricas</font>**

<p align="justify">
<code>StandardScaler</code> es una clase de la biblioteca <code>scikit-learn</code> en Python que se utiliza para estandarizar variables numéricas en un conjunto de datos. La estandarización es un paso común en el preprocesamiento de datos antes de aplicar técnicas de aprendizaje automático, ya que ayuda a que las variables tengan una escala común y elimina cualquier sesgo relacionado con la escala de las variables explicativas.


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

<p align="justify">
La clase <code>StandardScaler</code> implementa el método <code>fit_transform()</code> que se utiliza para ajustar y transformar los datos al mismo tiempo. Este método calcula la media y la desviación estándar de cada variable del conjunto de datos y luego las utiliza para estandarizar cada registro. La estandarización se realiza restando la media a cada registro y dividiendolo por la desviación estándar, lo que resulta en variables con una media de cero y una desviación estándar de uno.

In [ ]:
X_scaled = scaler.fit_transform(X)
X_scaled

<p align="justify">
Para transformar los datos estandarizados (<code>numpy array</code>) a un <code>DataFrame</code> es posible utilizar la biblioteca el método <code>DataFrame()</code> de <code>Pandas</code>. En resumen, este método permite crear un <code>DataFrame</code> a partir de la matriz de datos estandarizados.

In [ ]:
X_scaled = pd.DataFrame(X_scaled, columns = X.columns)
X_scaled

 ## **División del conjunto de datos</font>**

<p align="justify">
División del conjunto de entrenamiento y prueba.
</p>


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(X_scaled, y, random_state=123)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

 ## **Ajuste del modelo con sklearn</font>**

<p align="justify">
La clase <code>LinearRegression</code> de <code>scikit-learn</code> proporciona una interfaz sencilla y eficiente para ajustar modelos de regresión lineal y realizar predicciones. Es ampliamente utilizada en problemas de regresión en los que se busca modelar y predecir una variable dependiente a partir de variables independientes mediante una relación lineal.
<br>
<br>
En este caso, se intenta predecir el valor del <code>Gasto_anual</code> en función de las variables explicativas <code>Tiempo_sesión</code>, <code>Tiempo_app</code>,	<code>Tiempo_web</code> y <code>Años_miembro</code>.
<br>
<br>
Problema de regresión lineal múltiple.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model_1 = LinearRegression().fit(X_train, y_train)
print("")
print(f"intercept = {model_1.intercept_}")
print(f"coef = {model_1.coef_}")

Imprimimos el intercepto del modelo, que es el valor de y cuando todas las características **(X)** son cero.

Imprimimos los coeficientes asociados a cada característica en **X**. Estos coeficientes representan la relación entre cada característica y la variable de destino (**y**).

En un modelo de regresión lineal simple, esto sería la pendiente de la línea de regresión. En modelos más complejos con múltiples características, habrá un coeficiente para cada característica.

### **Interpretación**

$Gasto \ anual = 499.1+25.1 × Tiempo \ sesión + 38.6 × Tiempo \ app + 0.8 × Tiempo \ web + 61.6 × Años\ miembro$

**Interpretación de los coeficientes:**

Manteniendo las otras características fijas,
+ un incremento de 1 unidad en `Tiempo_sesión` está asociado con un incremento de 25.14 en `Gasto_anual`,
+ un incremento de 1 unidad en `Tiempo_app` está asociado con un incremento de 38.6 en `Gasto_anual`,
+ un incremento de 1 unidad en `Tiempo_web` está asociado con un incremento de 0.77 en `Gasto_anual`y
+ un incremento de 1 unidad en `Años_miembro` está asociado con un incremento de 61.59 en `Gasto_anual`.

### **Predicción y evaluación del modelo**

Una vez ajustado el modelo, procedemos a realizar predicciones y evaluar su desempeño

In [ ]:
prediction = model_1.predict(X_test)
prediction[:5]

In [ ]:
tabla = pd.DataFrame({"Prediccion":prediction,
                      "Real":y_test,
                      "Residuos": (y_test-prediction),
                      })
tabla.head()

In [ ]:
model_1.score(X_test, y_test)

La función **score** en scikit-learn se utiliza para evaluar el rendimiento de un modelo en datos de prueba. En el contexto de la regresión lineal, la función score devuelve el coeficiente de determinación (R^2).

El coeficiente de determinación (R^2) es una medida estadística que indica qué tan bien el modelo se ajusta a los datos de prueba. Toma valores entre 0 y 1, donde 1 indica un ajuste perfecto y 0 indica que el modelo no explica nada de la variabilidad de los datos de prueba.

En términos simples, un R^2 más alto significa que el modelo es capaz de explicar una mayor proporción de la variabilidad en los datos de prueba. Sin embargo, es importante tener en cuenta que un R^2 alto en los datos de prueba no garantiza un buen rendimiento en datos nuevos y no vistos. La evaluación del rendimiento del modelo debe hacerse con una combinación de métricas y validación cruzada.

In [ ]:
from sklearn import metrics
SMSE = round(metrics.mean_squared_error(y_test, prediction,squared=False),2) #Error cuadrático medio
MAE = round(metrics.mean_absolute_error(y_test, prediction),2) #Error absoluto medio
R2 = round(metrics.r2_score(y_test, prediction),4)
print("")
print("SMSE: {}".format(SMSE))
print("MAE: {}".format(MAE))
print("R2: {}".format(R2))

Este código está calculando diversas métricas de evaluación del rendimiento de un modelo de regresión.

- **`SMSE` (Error cuadrático medio):** Esta métrica mide la raíz cuadrada del error cuadrático medio entre las predicciones del modelo y los valores reales del conjunto de prueba. Es una medida de cuánto se espera que varíen las predicciones del modelo con respecto a los valores reales. Un SMSE más bajo indica un mejor rendimiento del modelo.

- **`MAE` (Error absoluto medio):** Esta métrica mide el promedio de las diferencias absolutas entre las predicciones del modelo y los valores reales en el conjunto de prueba. Es menos sensible a los valores atípicos que el error cuadrático medio. Un MAE más bajo indica un mejor rendimiento del modelo.

- **`R2` (Coeficiente de determinación):** Esta métrica, ya mencionada en una respuesta anterior, indica la proporción de la varianza en la variable dependiente que es predecible a partir de las variables independientes. Toma valores entre 0 y 1, donde 1 indica un ajuste perfecto. Un R2 más alto indica un mejor rendimiento del modelo.


## **Creación del Pipeline**

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
model = make_pipeline(StandardScaler(), LinearRegression())
model

Un pipeline es una forma de simplificar el flujo de trabajo en aprendizaje automático al combinar varias operaciones en un solo objeto. En este caso, el pipeline consta de dos pasos: **StandardScaler()** y **LinearRegression()**.

El pipeline encapsula estos dos pasos en un solo objeto llamado **model**.

Ahora, puedes entrenar y utilizar este modelo de manera más sencilla, ya que el pipeline se encarga automáticamente de aplicar la estandarización antes de ajustar el modelo de regresión lineal.

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
cv_results = cross_validate(model, X, y, cv=5)
cv_results

 # **La necesidad de validación cruzada</font>**

<p align="justify">
En el ejemplo anterior, se dividieron los datos originales en un conjunto de datos de entrenamiento y un conjunto de datos de prueba. La puntuación de la generalización del modelo entonces, dependerá en general de la forma en que se hace esa división.
<br><br>
Una desventaja de hacer una sola división, es que no da información sobre esta variabilidad. Otro inconveniente, en un entorno donde la cantidad de datos es pequeña, es que los datos disponibles para el entrenamiento y los datos disponibles para la prueba serán aún más pequeña después de esa división.
<br><br>
Entonces, podemos utilizar la validación cruzada.
<br><br>
La validación cruzada consiste en repetir el procedimiento de manera que el conjunto de datos de entrenamiento y el conjunto de datos de prueba sean diferentes en cada repetición. Las métricas de rendimiento de la generalización del modelo se recopilan para cada repetición. Como resultado, se puede evaluar la variabilidad del rendimiento del modelo.
<br><br>

Por ahora, vamos a usar la estrategia <code>K-fold</code> que implica que todo el conjunto de datos se divide en $K$ particiones. El procedimiento de ajuste <code>fit</code> y la evaluación <code>score</code> se repite $K$ veces donde en cada iteración $K - 1$ las particiones se usan para ajustar el modelo.
<br><br>
La siguiente figura ilustra esta estrategia <code>K-fold</code>.
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/k-fold-001.png?raw=true" width="600">
</p>


<p align="justify">
Esta figura muestra el caso particular de la estrategia de validación cruzada <code>K-fold</code>
<br><br>
Para cada división de validación cruzada, el procedimiento entrena un clon del modelo en todos los puntos rojos y evalua la puntuación del modelo en los azules. Como se mencionó anteriormente, hay una variedad de diferentes validaciones cruzadas. Por lo tanto, la validación cruzada es computacionalmente intensiva porque requiere entrenar varios modelos, en vez de entrenar solo uno.
<br><br>
En <code>scikit-learn</code>, la función <code>cross_validate</code> permite realizar la validación cruzada y necesita pasar el modelo, los datos y la variable objetivo. El parámetro <code>cv</code> define la estrategia de división, es decir, en cuanto se divide...
</p>


In [ ]:
scores = cv_results["test_score"]
print("")
print("El R2 mediante cross-validation es: "
      f"{scores.mean():.3f} ± {scores.std():.3f}")

*El modelo con todas las variables numéricas como predictores tiene un $𝑅^2$  muy alto (0.984), es capaz de explicar el 98.4% de la variabilidad observada en el* `Gasto_anual`.

 # **Conclusiones</font>**

<p align="justify">
A través de este ejercicio nosotros:<br>

*  Estudiamos la correlación lineal entre variables.

*  Procedimos a estandarizar las variables explicativas numéricas con el fin de homogeneizar su escala.

*  Utilizamos la biblioteca <code>scikit-learn</code> para entrenar un modelo de regresión lineal múltiple.
<br>
*  Realizamos la predicción y evaluación, usando diferentes métricas, con un conjunto de prueba.
<br>
*  Implementamos un Pipeline y aplicamos la validación cruzada.
<br>


